<a href="https://colab.research.google.com/github/LatiefDataVisionary/feature-engineering-college-task/blob/main/feature_engineering_w9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.decomposition import PCA

In [6]:
# Data simulasi
data = {
    "Luas Rumah (m2)": [120, 250, 85, 190],
    "Jumlah Kamar Tidur": [3, 5, 2, 4],
    "Lokasi": ["Kota A", "Kota B", "Kota A", "Kota C"],
    "Tahun Dibangun": [2000, 2010, 1995, 2020],
    "Harga Rumah (Ribu $)": [300, 450, 200, 500],  # Target
}

In [7]:
# DataFrame awal
df = pd.DataFrame(data)
df

,Luas Rumah (m2),Jumlah Kamar Tidur,Lokasi,Tahun Dibangun,Harga Rumah (Ribu $)
0,120,3,Kota A,2000,300
1,250,5,Kota B,2010,450
2,85,2,Kota A,1995,200
3,190,4,Kota C,2020,500


In [8]:
# --- Step 1: Feature Extraction ---
# Mengambil fitur relevan dari data awal
features = df[["Luas Rumah (m2)", "Jumlah Kamar Tidur", "Lokasi", "Tahun Dibangun"]]
target = df["Harga Rumah (Ribu $)"]

In [9]:
# --- Step 2: Feature Construction ---
# Menambahkan fitur baru: Umur Rumah
# Informasi tambahan: Fitur ini dihitung dari tahun sekarang
df["Umur Rumah"] = 2024 - df["Tahun Dibangun"]

In [10]:
# --- Step 3: Transformasi Fitur Kategorikal ---
# Menggunakan One-Hot Encoding untuk fitur kategorikal "Lokasi"
encoder = OneHotEncoder(sparse_output=False)  # Menghasilkan array dense
encoded_locations = encoder.fit_transform(df[["Lokasi"]])

# Membuat DataFrame untuk hasil encoding
encoded_location_df = pd.DataFrame(encoded_locations, columns=encoder.get_feature_names_out(["Lokasi"]))

# Menggabungkan hasil encoding ke DataFrame asli
df = pd.concat([df, encoded_location_df], axis=1)

# Menghapus fitur "Lokasi" dan "Tahun Dibangun" karena sudah diwakili oleh fitur baru
df.drop(["Lokasi", "Tahun Dibangun"], axis=1, inplace=True)

In [11]:
# --- Step 4: Scaling and Normalization ---
# Menggunakan StandardScaler untuk mengubah fitur ke skala standar (mean=0, std=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[["Luas Rumah (m2)", "Jumlah Kamar Tidur", "Umur Rumah"]])

# Menambahkan fitur yang sudah diskalakan kembali ke DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=["Luas Rumah (scaled)", "Jumlah Kamar (scaled)", "Umur Rumah (scaled)"])
df = pd.concat([df, scaled_df], axis=1)
df

,Luas Rumah (m2),Jumlah Kamar Tidur,Harga Rumah (Ribu $),Umur Rumah,Lokasi_Kota A,Lokasi_Kota B,Lokasi_Kota C,Luas Rumah (scaled),Jumlah Kamar (scaled),Umur Rumah (scaled)
0,120,3,300,24,1.0,0.0,0.0,-0.647806,-0.447214,0.650945
1,250,5,450,14,0.0,1.0,0.0,1.393765,1.341641,-0.390567
2,85,2,200,29,1.0,0.0,0.0,-1.197460,-1.341641,1.171700
3,190,4,500,4,0.0,0.0,1.0,0.451501,0.447214,-1.432078


In [12]:
# --- Step 5: Log Transformation ---
# Mengurangi skewness pada target (Harga Rumah)
df["Log Harga Rumah"] = np.log1p(df["Harga Rumah (Ribu $)"])

In [13]:
# --- Step 6: Dimensionality Reduction (PCA) ---
# Menerapkan PCA pada fitur numerikal untuk mereduksi dimensi
pca = PCA(n_components=2)
numerical_features = df[["Luas Rumah (scaled)", "Jumlah Kamar (scaled)", "Umur Rumah (scaled)"]]
pca_features = pca.fit_transform(numerical_features)

# Menambahkan fitur hasil PCA ke DataFrame
pca_df = pd.DataFrame(pca_features, columns=["PCA1", "PCA2"])
df = pd.concat([df, pca_df], axis=1)

In [14]:
# --- Step 7: Hasil Akhir ---
# Menghapus fitur yang tidak relevan setelah transformasi
df.drop(["Luas Rumah (m2)", "Jumlah Kamar Tidur", "Umur Rumah", "Luas Rumah (scaled)",
         "Jumlah Kamar (scaled)", "Umur Rumah (scaled)"], axis=1, inplace=True)

# Menampilkan hasil akhir dataset
print("Dataset setelah semua tahap Feature Engineering:")
print(df)

Dataset setelah semua tahap Feature Engineering:
   Harga Rumah (Ribu $)  Lokasi_Kota A  Lokasi_Kota B  Lokasi_Kota C  \
0                   300            1.0            0.0            0.0   
1                   450            0.0            1.0            0.0   
2                   200            1.0            0.0            0.0   
3                   500            0.0            0.0            1.0   

   Log Harga Rumah      PCA1      PCA2  
0         5.707110 -1.002836  0.131332  
1         6.111467  1.840557  0.711579  
2         5.303305 -2.143854  0.022332  
3         6.216606  1.306133 -0.865243  
